In [1]:
import argparse
import os
import ruamel.yaml as yaml
import numpy as np
import random
import time
import datetime
import json
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.distributed as dist
from torch.utils.data import DataLoader

from models.blip_pretrain import blip_pretrain
import utils
from utils import warmup_lr_schedule, step_lr_schedule
from data import create_dataset, create_sampler, create_loader, create_forget_loader

/home/gloriel621/.conda/envs/blip/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'create_unlearn_loader' from 'data' (/home/gloriel621/BLIP/data/__init__.py)

In [ ]:
# Simulating command line arguments
class Args:
    config = './configs/unlearn_pretrain.yaml'
    output_dir = 'output/Pretrain'
    checkpoint = ''
    evaluate = False
    device = 'cuda'
    seed = 42
    world_size = 1
    dist_url = 'env://'
    distributed = True

args = Args()

# Load configuration
with open(args.config, 'r') as file:
    config = yaml.load(file, Loader=yaml.Loader)

# Ensure the output directory exists
Path(args.output_dir).mkdir(parents=True, exist_ok=True)

# Save configuration back to a file in the output directory
config_path = os.path.join(args.output_dir, 'config.yaml')
with open(config_path, 'w') as file:
    yaml.dump(config, file)

print("Configuration loaded and saved at:", config_path)


Configuration loaded and saved at: output/Pretrain/config.yaml


In [ ]:
config

{'train_file': ['./json_pretrain/coco_updated.json'],
 'laion_path': '',
 'pretrained': 'output/Pretrain/checkpoint_19.pth',
 'vit': 'base',
 'vit_grad_ckpt': False,
 'vit_ckpt_layer': 0,
 'image_size': 224,
 'batch_size': 20,
 'queue_size': 57600,
 'alpha': 0.4,
 'weight_decay': 0.05,
 'init_lr': 0.0003,
 'min_lr': 1e-06,
 'warmup_lr': 1e-06,
 'lr_decay_rate': 0.9,
 'max_epoch': 2,
 'warmup_steps': 3000}

In [ ]:
utils.init_distributed_mode(args)    
    
device = torch.device(args.device)

# fix the seed for reproducibility
seed = args.seed + utils.get_rank()
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
cudnn.benchmark = True

#### Dataset #### 
#### Dataset #### 
print("Creating dataset")

# create unlearn and retain loaders.
dataset = create_dataset('pretrain', config, min_scale=0.2)
print('number of training samples: %d'%len(dataset))

num_tasks = utils.get_world_size()
global_rank = utils.get_rank()            
samplers = create_sampler([dataset, dataset], [True, True], num_tasks, global_rank) 

loaders = create_unlearn_loader(dataset, config['batch_size'], 4, None, samplers, forget_percentage=0.01)
retain_loader = loaders['retain']
unlearn_loader = loaders['unlearn']   

Not using distributed mode
Creating dataset
loading ./json_pretrain/coco_updated.json
number of training samples: 566747


In [ ]:
len(retain_loader)

28337

In [ ]:
len(unlearn_loader)

28337

In [ ]:
check_indices(dataset, retain_indices)
check_indices(dataset, unlearn_indices)